In [65]:
import json
import os
import numpy as np
import xml.etree.ElementTree as ET


for idx, (root, dirs, files) in enumerate(os.walk('json_test')):
    Imagelist = [Image for Image in files if Image.lower().endswith(".png")]
    Jsonlist = [Json for Json in files if Json.lower().endswith(".json")]
    
    for num in range(len(Imagelist)):
        Image = Imagelist[num]
        Json = Jsonlist[num]
         
        with open(os.path.join(root, Json),"rb") as jsonFile:
            Objects = json.load(jsonFile)
        
            # makeXml     
            xml_root = ET.Element('annotation')
            ET.SubElement(xml_root, 'folder').text = root
            ET.SubElement(xml_root, 'filename').text = Image
            ET.SubElement(xml_root, 'path').text = os.path.join(os.getcwd(), root, Image)
                
                
                        
            source = ET.SubElement(xml_root, 'source')
            ET.SubElement(source, 'database').text = 'Unknown'
                        
                        
            size = ET.SubElement(xml_root, 'size')
            ET.SubElement(size, 'width').text = str(1920)
            ET.SubElement(size, 'height').text = str(1200)
            ET.SubElement(size, 'depth').text = str(3)
                            
            ET.SubElement(xml_root, 'segmented').text = '0'
                        
                        #xmin, ymin, xmax, ymax
            total_object = len(Objects['annotations'])
            #object = ET.SubElement(xml_root, 'object')
            for object_idx in range(total_object):
                c_object = ET.SubElement(xml_root, 'object')
                
                if  'categories' not in Objects.keys() :
                    pass
                            
                else : 
                    ET.SubElement(c_object, 'name').text = Objects['categories'][object_idx]['label']
                    
                    if Objects['categories'][object_idx]['label'] == '자동차_일반자동차' or '자동차_SUV' or '자동차_특수차량' or '자동차_기타자동차':
                        Objects['categories'][object_idx]['label'] == 'car'
                        
                    elif  Objects['categories'][object_idx]['label'] == '자동차_버스':
                        Objects['categories'][object_idx]['label'] == 'bus'
                        
                    elif  Objects['categories'][object_idx]['label'] == '자동차_트럭':
                         Objects['categories'][object_idx]['label'] == 'truck'
                         
                    elif Objects['categories'][object_idx]['label'] == '이륜차_이륜차' or '이륜차_사륜차' or '이륜차_전동휠' or '이륜차_PM(교통약자 이동장치)' or '이륜차_기타' or '보행자_자전거':
                        Objects['categories'][object_idx]['label'] == '2wheels'
                        
                    elif Objects['categories'][object_idx]['label'] == '보행자_보행자':
                        Objects['categories'][object_idx]['label'] =='person'
                    
                    elif Objects['categories'][object_idx]['label'] == '신호등_적색' or '신호등_황색' or '신호등_녹색' or '신호등_화살표' or '신호등_적색&화살표' or '신호등_황색&화살표' or '신호등_녹색&화살표' or '신호등':
                        Objects['categories'][object_idx]['label'] == 'traffic_light'
                    else : pass
                    
                    ET.SubElement(c_object, 'pose').text = 'Unspectified'
                    ET.SubElement(c_object, 'truncated').text = '0'
                    ET.SubElement(c_object, 'difficult').text = '0'
 
                anno = Objects["annotations"][object_idx]
                 
                if  'bbox' not in anno.keys() :
                    pass
                
                else : 
                    points = Objects['annotations'][object_idx]['bbox']
                    npoints = np.array(points)    
                    rpoints = np.reshape(npoints,-1)
                        
                    
                    x = rpoints[0]
                    y = rpoints[1]
                                        
                    x_min = x
                    x_max = x + rpoints[2]
                                        
                    y_min = y
                    y_max = y + rpoints[3]          
                        
                
                    bbox = ET.SubElement(c_object, 'bnbox')
                    ET.SubElement(bbox, 'xmin').text = str(x_min)
                    ET.SubElement(bbox, 'ymin').text = str(y_min)
                    ET.SubElement(bbox, 'xmax').text = str(x_max)
                    ET.SubElement(bbox, 'ymax').text = str(y_max)
                
        tree = ET.ElementTree(xml_root)
        tree.write(os.path.join(os.getcwd(), root, Image[:-4] + '.xml'))  
        